Take model used in v3 and try one hot encoding output

In [1]:
# import libraries
import numpy as np
import csv
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelBinarizer

In [2]:
# Read in the data
# Create the training and test sets, skipping the header
# row with [1:]
dataset = np.genfromtxt(open('train.csv', 'r'),
                        delimiter=',', dtype='f8')[1:]
target = [x[0] for x in dataset]
train = [x[1:]/255 for x in dataset]
test = np.genfromtxt(open('test.csv', 'r'),
                     delimiter =',', dtype='f8')[1:]
test = [x/255 for x in test]

In [3]:
# 1 hot encode target labels
target_ones = label_binarize(target, classes=[0,1,2,3,4,5,6,7,8,9])

In [4]:
lb = LabelBinarizer()
lb.fit(target)
print(lb.classes_)
target_1 = lb.transform(target)
target_1

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]


array([[0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [5]:
# set model use parameters from v3
mlp = MLPClassifier(
    early_stopping=True, 
    hidden_layer_sizes=(2000),
    max_iter=200,
    learning_rate='adaptive'
)


In [ ]:
# hyperparameters
param_grid = {
    'hidden_layer_sizes': [
        (2000)
    ],
    'max_iter': [200, 400],
    'learning_rate': ['adaptive'] 
}

clf = GridSearchCV(mlp, param_grid)
clf.fit(train, target_ones)
print('Best score: {}'.format(clf.best_score_))
print('Best parameters: {}'.format(clf.best_params_))

Previous result:
- Best score: 0.973357142857
- Best parameters: {'hidden_layer_sizes': 800}

- Best score: 0.974071428571
- Best parameters: {'hidden_layer_sizes': 2000}

- Best score: 0.975523809524
- Best parameters: {'learning_rate': 'constant', 'max_iter': 200, 'hidden_layer_sizes': 2000}

In [6]:
mlp.fit(train, target_ones)
mlp.score(train, target_ones)
#clf.score(train, target_ones)

0.99595238095238092

In [8]:
# predictions for test set
predictions = mlp.predict(test)

In [11]:
# transform back to labels
results = lb.inverse_transform(predictions)

In [12]:
# save predictions to file
output = csv.writer(open('results-v4.csv', 'wt'))
output.writerow(["ImageId", "Label"])
for i, res in enumerate(results):
    output.writerow([i+1, int(res)])